# <div align="center"> Part 1 : Extracting historical tweets </div>

<a id='sec1'></a>
## 1. Twitter Historical Data Receiving

The goal for our project is to find the relationship between twitter sentimental score and stock price for that company in real time, so the first thing we need to do is to receive the  historical Twitter data and then train our model on the dataset. 

We need to set up the Twitter API credential first, in order to receive the data from Twitter. We directly go to Twitter Developers (https://dev.twitter.com/apps) and create an application, and after the applicaiton is created, we can get our Access Token and Customer Key. 

We use the following code to get the historical data. The following is an example of code to show how we get the Tweets about "@Target", and followed by some of the results of the Tweets we received. 

Replace the credentials with your credentials you got from Twitter Developer. Here in our code, we stored the Tweets we got into a sperate csv file for the use in the future, so if you don't want to store it, you can just remove this part of code. 

In [2]:
import tweepy
import csv

access_token = "797512809810640896-pcpnw55PYPqtRnanoUWyvO5xUMIOEVU"
access_token_secret = "qxhDMlJClxuBc3KxAiYsk3VgayDsuNDbD76QqVogf11N7"
consumer_key = "zO5qI6mjCzF4j3bWNIrlCAaeq"
consumer_secret = "8ePrHNmIRPtbRypdSxmhdOAZHFgb93Oa0zaab0NAJsQDpRklId"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Open/Create a file to append data
csvFile = open('target_tweets.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

Replace the 'q' value with the key words you want to search for, and here we use "@Target" as our key word. It will offer us all the Tweets with the keyword "@Target" in English since "2016-06-30". The code will start from the moment you start to run the code, and go back from now to get the maximum 18,000 Tweets per 15 mins. So if before the time you want it to stop, the number of Tweets you got has already achieved 18,000 the program will stop. "count" can maximum be 100, and it means that the largest number of Tweets we can get for one token, if you set it to 100, the maximum you will recieve the 18,000 records in 15 mins. 

In [ ]:
for tweet in tweepy.Cursor(api.search,q="@Target",count=1,\
                           lang="en",\
                           since_id=2016-06-30).items():
    print tweet.created_at, tweet.text
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

Output Example:

2016-12-11 21:19:26 RT @johnlegend: Head to @target to get the exclusive deluxe version of my album #DARKNESSANDLIGHT with 3 extra songs! https://t.co/pmJQcRkH…
2016-12-11 21:19:17 @Target pharmacy v. @CVS_Extra  miss that @Target pharm! https://t.co/CML1NO1vI2
2016-12-11 21:18:34 RT @reignmurphy: @Target as someone with OCD I'd really appreciate it if you didn't sell my illness as a fashion statement https://t.co/34e…

# <div align="center"> Part 2 : Vader sentiment analysis </div>

<a id='sec1'></a>
## 2. Sentiment Analysis With Historical Twitter

In order to do the sentiment analysis based on the historical Tweets, we read the csv file we just wrote, and save it as tweetRDD.

In [ ]:
import pandas as pd
from datetime import datetime
tweetRDD = sc.textFile('file:///Users/MyProject/target_tweets.csv') 

Save the tweetRDD into dataframe format and call it allTweet. 

In [ ]:
allTweet=pd.DataFrame(tweetRDD.collect())

We use the VADER Sentiment to do the Sentiment analysis, so we first need to install the package called 'vaderSentiment'. The documentation for VADER Sentiment can be found <a>here</a>.

Then we start to do the Sentiment analysis to each row from the csv file, and write the result to another csv file then use spark context to get the results. With the analyzer we can get 4 sentimental scores, that are negetive score, positive score, neutural score and a compound socre for other three scores.

In [ ]:
import vaderSentiment
if __name__ == '__main__':
    # --- examples -------
    sentences = [];
    date = [];
    for index, row in allTweet.iterrows():
        sentences.append(row[0])
        a = row[0].split(',')
        date.append(a[0])
    analyzer = SentimentIntensityAnalyzer()
    compoundscore = [];
    posscore = [];
    negscore = [];
    neuscore = [];
    for sentence in sentences:
        vs = analyzer.polarity_scores(sentence)
        compoundscore.append(vs['compound'])
        posscore.append(vs['pos'])
        negscore.append(vs['neg'])
        neuscore.append(vs['neu'])
        #print("{:-<65} {}".format(sentence, str(vs)))
        
    ofile = open('/Users/MyProject/target_tweets_final_score1.csv','wb')
    writer = csv.writer(ofile)
    
    for i in range(0,len(sentences)):
            print date[i]
            print compoundscore[i]
            print posscore[i]
            print negscore[i]
            print neuscore[i]

            writer.writerow([date[i].encode('utf-8'), \
                             compoundscore[i],posscore[i],negscore[i],neuscore[i]])
    ofile.close()

Output example:

12/10/2016 17:16
0.561
0.333
0.148
0.519

Read the data from the csv file, and then use mapping function to split the data into several parts and then map them into pair RDD with the date as the key and the four scores as the values.

In [ ]:
amazon_with_score = sc.textFile('file:///Users/MyProject/target_tweets_final_score1.csv')
amazonScoreRDD = amazon_with_score.map(lambda line: line.split(',')) \
.map(lambda field:(field[0],(field[1],field[2],field[3],field[4])))
amazonScoreRDD.take(1)

Output example:
[(u'12/10/2016 17:16', (u'0.561', u'0.333', u'0.148', u'0.519'))]

Then map the datetime and the four sentiment scores to pairs for future calculation of mean. Because we will get more than one tweets in one min, but we want to do the analysis just on minutes level, so we want to calculate an average score for all the tweets in every minutes. 

In [ ]:
amazonScoreRDD1 = amazonScoreRDD.map(lambda f:(str(f[0])[0:16],((float(f[1][0]),1), \
 (float(f[1][1]),1),(float(f[1][2]),1),(float(f[1][3]),1))))
compoundScoreRDD1 = amazonScoreRDD1.map(lambda f: (str(f[0]),f[1][0]))
posScoreRDD1 = amazonScoreRDD1.map(lambda f: (str(f[0]),f[1][1]))
negScoreRDD1 = amazonScoreRDD1.map(lambda f: (str(f[0]),f[1][2]))
neuScoreRDD1 = amazonScoreRDD1.map(lambda f: (str(f[0]),f[1][3]))

In [ ]:
amazonScoreRDD1.take(1)[0][1]

Output example:
((0.561, 1), (0.333, 1), (0.148, 1), (0.519, 1))

Then use reduceByKey and mapValues to calculate the average for Tweets in every minutes.

In [ ]:
compoundScoreRDD2=compoundScoreRDD1.reduceByKey(lambda v1,v2:(v1[0]+v2[0],v1[1]+v2[1])).mapValues(lambda v:v[0]/v[1])
posScoreRDD2 = posScoreRDD1.reduceByKey(lambda v1,v2:(v1[0]+v2[0],v1[1]+v2[1])).mapValues(lambda v:v[0]/v[1])
negScoreRDD2 = negScoreRDD1.reduceByKey(lambda v1,v2:(v1[0]+v2[0],v1[1]+v2[1])).mapValues(lambda v:v[0]/v[1])
neuScoreRDD2 = neuScoreRDD1.reduceByKey(lambda v1,v2:(v1[0]+v2[0],v1[1]+v2[1])).mapValues(lambda v:v[0]/v[1])

In [ ]:
compoundScoreRDD2.take(2)

Output example: 
[('12/8/2016 2:19', 0.5882333333333333), ('12/7/2016 21:27', 0.1851)]

In [ ]:
# Join the rdds together
amazonScoreRDD3 = compoundScoreRDD2.join(posScoreRDD2).map(lambda l: (l[0],l[1][0],l[1][1]))
# Join the rdds together
neuNegScoreRDD3 = neuScoreRDD2.join(negScoreRDD2).map(lambda l: (l[0],l[1][0],l[1][1]))

After We have the rdds joint, we can change that to dataframe and do the further programming.

In [ ]:
neuNeg = pd.DataFrame(neuNegScoreRDD3.collect())
neuNeg.columns = ['date', 'neu','neg']
a = pd.DataFrame(amazonScoreRDD3.collect())
a.columns = ['date', 'com','pos']
whole = pd.merge(a, neuNeg, how='inner', on='date')

Write the final file to the csv file for further usage.

In [ ]:
whole.to_csv('/Users/MyProject/target_tweets_complete_score_mins.csv')

## VADER Sentiment Analysis

### Introduction

Sentiment analysis aims to determine the attitude of a speaker or a writer with respect to some topic or the overall contextual polarity of a document. It is useful to a wide range of problems that are of interest to human-computer interaction practitioners and researchers, as well as those from fields such as sociology, marketing and advertising, psychology, economics, and political science. The inherent nature of microblog content - such as those observed on Twitter and Facebook - poses serious challenges to practical applications of sentiment analysis. Some of these challenges stem from the sheer rate and volume of user generated social content, combined with the contextual sparseness resulting from shortness of the text and a tendency to use abbreviated language conventions to express sentiments.

VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media. It is fully open-sourced under the [MIT License]. Vader  use a combination of qualitative and quantitative methods to produce, and then empirically validate, a gold-standard sentiment lexicon that is especially attuned to microblog-like contexts. Then they combine these lexical features with consideration for five generalizable rules that embody grammatical and syntactical conventions that humans use when expressing or emphasizing sentiment intensity. 
The VADER lexicon performs exceptionally well in the social media domain. The correlation coefficient shows that VADER performs as well as individual human raters. VADER actually even outperforms individual human raters at correctly classifying the sentiment of tweets into positive, neutral, or negative classes.

VADER retains (and even improves on) the benefits of traditional sentiment lexicons like LIWC: it is bigger, yet just as simply inspected, understood, quickly applied (without a need for extensive learning/training) and easily extended. Like LIWC (but unlike some other lexicons or machine learning models), the VADER sentiment lexicon is gold-standard quality and has been validated by humans.

VADER distinguishes itself from LIWC in that it is more sensitive to sentiment expressions in social media contexts while also generalizing more favorably to other domains.

### Installation

There are a couple of ways to install and use VADER sentiment. The simplest is to use the command line to do an installation from PyPI using pip.

### Example
The below snippet shows how to use vaderSentiment in Python code.
from vaderSentiment import sentiment as vaderSentiment

In [ ]:
sentences =
["VADER is smart, handsome, and funny.",       # positive sentence example
 "VADER is smart, handsome, and funny!",       # punctuation emphasis handled correctly 
]
analyzer = vaderSentiment.SentimentIntensityAnalyzer()
for sentence in sentences:
       print sentence
       vs = analyzer.polarity_scores(sentence)
       print "\n\t" + str(vs)

The output provides the polarity and intensity of the inputs. The compound in the output provides the sentiment intensity and will be in the range of -1 (Extremely negative) to 1 (Extremely positive).  Based on the compound value, you can create more ranges to define more categorical values for sentiment(Extremely positive, positive, neutral, negative, Extremely negative).

<p>VADER is smart, handsome, and funny.</p>
<p>  {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}</p>
<p>VADER is smart, handsome, and funny!</p>
<p>  {'neg': 0.0, 'neu': 0.248, 'pos': 0.752, 'compound': 0.8439}</p>
<p>VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!</p>
<p>  {'neg': 0.0, 'neu': 0.294, 'pos': 0.706, 'compound': 0.9469}</p>
<p>A really bad, horrible book.</p>
<p> {'neg': 0.791, 'neu': 0.209, 'pos': 0.0, 'compound': -0.8211}</p>

Use Vader, we can get a score for each tweet we get. Score contains 4 parts: positive score, negative score, neutral score, compound score. If the compound score > 0, that means this tweet is positive, if the compound score < 0, that means this tweet is negative. Compound score is normalized to the range [-1,1].
We use VADER sentiment analysis to all the tweets we get for company and save the sentiment score along with the date, tweet text to a csv for further data processing. 

### Reference 
[1]C.J. Hutto and Eric Gilbert,”VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text”,Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014